In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from scrape import extract_players
from scrape import extract_year
from scrape import make_dists
from scrape import extract_age
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
import re
import matplotlib.pyplot as plt
from scrape import read_targets
from scrape import clean_name
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import scale
from scrape import run_svr_2023
from scrape import run_xgb_2023
from scrape import xgb_2023
from scrape import get_data

In [2]:
results = run_svr_2023('wr', 'PPR', pca=True)
results['projections'].head(50)

justin-jefferson
tyreek-hill
davante-adams
stefon-diggs
ceedee-lamb
aj-brown
amonra-stbrown
jaylen-waddle
devonta-smith
amari-cooper
jamarr-chase
christian-kirk
tyler-lockett
terry-mclaurin
brandon-aiyuk
dk-metcalf
mike-evans
tee-higgins
chris-godwin
michael-pittman
garrett-wilson
jerry-jeudy
cooper-kupp
dj-moore-wr
chris-olave
zay-jones
juju-smith-schuster
diontae-johnson
jakobi-meyers
adam-thielen
drake-london
mike-williams-wr
curtis-samuel
tyler-boyd
allen-lazard
gabriel-davis
josh-palmer
deebo-samuel
donovan-peoples-jones
george-pickens
christian-watson
keenan-allen
courtland-sutton
marquise-brown
kj-osborn
mack-hollins
deandre-hopkins
parris-campbell
brandin-cooks
richie-james
jahan-dotson
darius-slayton
russell-gage
marquez-valdesscantling
isaiah-mckenzie
marvin-jones
kalif-raymond
devin-duvernay
robert-woods
chris-moore
noah-brown
greg-dortch
deandre-carter
alec-pierce
olamide-zaccheaus
michael-gallup
chase-claypool
josh-reynolds
demarcus-robinson
devante-parker
romeo-doubs
darn

c:\Users\xabbo\Desktop\fantasy_scrape\scrape.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ngs_data['name'] = ngs_data['PLAYER NAME'].apply(lambda x: clean_name(x))
c:\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Done for 2022
cooper-kupp
davante-adams
deebo-samuel
justin-jefferson
jamarr-chase
tyreek-hill
stefon-diggs
diontae-johnson
mike-evans
hunter-renfrow
keenan-allen
mike-williams-wr
jaylen-waddle
dk-metcalf
chris-godwin
tyler-lockett
michael-pittman
dj-moore-wr
ceedee-lamb
brandin-cooks
amonra-stbrown
marquise-brown
darnell-mooney
tee-higgins
terry-mclaurin
christian-kirk
amari-cooper
adam-thielen
jakobi-meyers
devonta-smith
tyler-boyd
aj-brown
kendrick-bourne
marvin-jones
brandon-aiyuk
van-jefferson
chase-claypool
russell-gage
cole-beasley
kj-osborn
aj-green
tim-patrick
marquez-callaway
courtland-sutton
cedrick-wilson
deandre-hopkins
allen-lazard
mecole-hardman
robby-anderson
elijah-moore
robert-woods
jarvis-landry
kalif-raymond
emmanuel-sanders
odell-beckham
laviska-shenault
byron-pringle
gabriel-davis
antonio-brown
braxton-berrios
quez-watkins
deonte-harty
donovan-peoples-jones
jamison-crowder
bryan-edwards
rondale-moore
nick-westbrook
zay-jones
corey-davis
devante-parker
rashod-batem

c:\Users\xabbo\Desktop\fantasy_scrape\scrape.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ngs_data['name'] = ngs_data['PLAYER NAME'].apply(lambda x: clean_name(x))
c:\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Done for 2021
davante-adams
tyreek-hill
stefon-diggs
deandre-hopkins
calvin-ridley
justin-jefferson
dk-metcalf
tyler-lockett
allen-robinson
adam-thielen
mike-evans
aj-brown
robert-woods
keenan-allen
amari-cooper
juju-smith-schuster
brandin-cooks
marvin-jones
robby-anderson
terry-mclaurin
diontae-johnson
ceedee-lamb
chase-claypool
curtis-samuel
dj-moore-wr
cooper-kupp
cole-beasley
tee-higgins
tyler-boyd
corey-davis
chris-godwin
will-fuller
jarvis-landry
nelson-agholor
brandon-aiyuk
marquise-brown
russell-gage
michael-gallup
jamison-crowder
devante-parker
emmanuel-sanders
ty-hilton
sterling-shepard
tim-patrick
jerry-jeudy
laviska-shenault
keelan-cole
mike-williams-wr
dj-chark
darnell-mooney
christian-kirk
julio-jones
jakobi-meyers
darius-slayton
marquez-valdesscantling
gabriel-davis
zach-pascal
greg-ward
hunter-renfrow
kendrick-bourne
mecole-hardman
josh-reynolds
rashard-higgins
david-moore
antonio-brown
travis-fulgham
damiere-byrd
aj-green
anthony-miller
demarcus-robinson
danny-amendola

c:\Users\xabbo\Desktop\fantasy_scrape\scrape.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ngs_data['name'] = ngs_data['PLAYER NAME'].apply(lambda x: clean_name(x))
c:\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Done for 2020
PCA Selected
Train dimensions (pre PCA):  (177, 33)
Test dimensions (pre PCA):  (91, 33)
Train dimensions (post PCA):  (177, 16)
Test dimensions (post PCA):  (91, 16)
Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-6.828 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-5.670 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-5.183 total time=   0.0s
[CV 4/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-4.683 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-5.759 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-6.638 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-5.380 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-4.991 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-4.549 total time=   0.0s
[CV 5/5] END .....C=0.1, gam

,name,proj fpts,class,2022 rank
0,justin-jefferson,20.192737,0,1
1,cooper-kupp,17.558308,1,23
2,jamarr-chase,17.521251,0,11
3,davante-adams,17.514616,0,3
4,tyreek-hill,17.454076,0,2
5,aj-brown,17.337447,0,6
6,stefon-diggs,17.210998,0,4
7,amonra-stbrown,16.735001,1,7
8,ceedee-lamb,16.321964,0,5
9,jaylen-waddle,15.770574,0,8


In [3]:
# results['projections'].to_csv('2022_wr_projections.csv')

In [4]:
dists2022 = results['2022_df']
dists2021 = results['2021_df']
dists2020 = results['2020_df']
proj = xgb_2023('wr', 'PPR', dists2020, dists2021, dists2022)
proj.head(50)

Index(['age', 'games_played', 'avg_rec', 'std_rec', 'avg_tgt', 'std_tgt',
       'avg_yds', 'std_yds', 'avg_ypr', 'std_ypr', 'avg_lg', 'std_lg',
       'avg_TD', 'std_TD', 'avg_rush', 'std_rush', 'avg_ryds', 'std_ryds',
       'avg_rypa', 'std_rypa', 'avg_rlg', 'std_rlg', 'avg_rTD', 'std_rTD',
       'TM TGT %', 'CUSHAverage Cushion (CUSH)', 'SEPAverage Separation (SEP)',
       'TAYAverage Targeted Air Yards (TAY)',
       'TAY%% Share of Team's Air Yards (TAY%)', 'YAC/RYAC/Reception (YAC/R)',
       'xYAC/RExpected YAC/Reception (EYAC/R)',
       '+/-Avg .YAC Above Expectation', 'class'],
      dtype='object') Index(['age', 'games_played', 'avg_rec', 'std_rec', 'avg_tgt', 'std_tgt',
       'avg_yds', 'std_yds', 'avg_ypr', 'std_ypr', 'avg_lg', 'std_lg',
       'avg_TD', 'std_TD', 'avg_rush', 'std_rush', 'avg_ryds', 'std_ryds',
       'avg_rypa', 'std_rypa', 'avg_rlg', 'std_rlg', 'avg_rTD', 'std_rTD',
       'TM TGT %', 'CUSHAverage Cushion (CUSH)', 'SEPAverage Separation (SEP)',
     

,name,proj fpts,class,2022 rank
0,justin-jefferson,19.267578,0,1
1,aj-brown,17.563293,0,6
2,jamarr-chase,17.413799,0,11
3,jaylen-waddle,17.317108,0,8
4,ceedee-lamb,17.137001,0,5
5,amonra-stbrown,16.959932,1,7
6,tyreek-hill,16.927258,0,2
7,stefon-diggs,16.847083,0,4
8,davante-adams,16.365912,0,3
9,cooper-kupp,16.017325,1,23


In [5]:
# proj.to_csv('2022_wr_projections_xbr.csv')